# need to code logic of this silly card game first ugh

In [1]:
# doing this on one screen on shitty hotel wifi is painful 
# start with defining rules -> dict (k,v), assigning arbitraty values (higher is better)
card_values = {'A': 100, 'K': 99, 'Q':98, 'J':97, 'T': 96, '9':95, '8':94, '7':93, '6':92, '5':91, '4':90, '3':89, '2':88}
# now need hand heirarcy, which is total value associated per hand (useful for comparison later)
def hand_outcomes(hand, card_heirarcy):
    # hand is a string
    hand_list = [*hand]
#     print(hand_list)
    hand_value = 0
    for card in hand_list:
#         print(card_heirarcy[card])
        hand_value += card_heirarcy[card]
    return hand_value 

In [2]:
#test 
hands = ['33332', '2AAAA']
outcomes = []
for hand in hands:
    outcomes.append(hand_outcomes(hand, card_values))

outcomes
# ok turns out due to the dumbass ordering rule (I didnt read that far yet lol) , this doesnt work so need to advise approach... 

[444, 488]

In [3]:
# i think we can still use above approach to determine orderings, just need to account for combinations
def hand_type(hand):
    h1 = [*hand]
    # first check 5 of kind
    if len(set(h1)) == 1:
        return 5
    elif len(set(h1)) == 2:
        # denotes both full house and 4 of kind so need to distinguish
        hand_count = {i:h1.count(i) for i in h1}
        # sort this dict by value
        if hand_count[max(hand_count, key=hand_count.get)] == 4:
            return 4
        else:
            return 3
    elif len(set(h1)) == 3:
        # need to distinguish between 3 of kind and 2 pair
        hand_count = {i:h1.count(i) for i in h1}
#         print(max(hand_count, key=hand_count.get))
        if hand_count[max(hand_count, key=hand_count.get)] == 3:
            return 2
        else:
            return 1
    elif len(set(h1)) == 4: # one pair
        return 0
    else: #high card
        return -1
'''
weird ordering but essentially
5 of kind : 5
4 of kind : 4
full house : 3 
3 of kind : 2
2 pair : 1
1 pair : 0
high card : -1

ps i know this can be optimized but im a lazy pos who is only driven by
solutions
and not optimization
'''

'\nweird ordering but essentially\n5 of kind : 5\n4 of kind : 4\nfull house : 3 \n3 of kind : 2\n2 pair : 1\n1 pair : 0\nhigh card : -1\n\nps i know this can be optimized but im a lazy pos who is only driven by\nsolutions\nand not optimization\n'

In [4]:
# now need to account for if you have same type, take ordering heirarchy
def compare_hands(hand1,hand2, card_heirarchy):
    h1t = hand_type([*hand1])
    h2t = hand_type([*hand2])
#     print(f"hand type 1: {h1t} hand type 2: {h2t}")
    if h1t == h2t: #hands are same type
        diff_ind = next((i for i, (el1,el2) in enumerate(zip([*hand1], [*hand2])) if el1 != el2),None)
        if diff_ind is not None:
            #get those values and compare 
            h1v = [*hand1][diff_ind]
            h2v = [*hand2][diff_ind]
            if card_heirarchy[h1v] > card_heirarchy[h2v]:
                return hand1
            else:
                return hand2
        else:
            return hand1
    elif h1t > h2t:
        return hand1
    else: 
        return hand2

In [5]:
compare_hands('33332','2AAAA',card_values)

'33332'

In [6]:
compare_hands('77888','77788',card_values)
# im a beast

'77888'

In [7]:
# ok now work in the example scenario
# need to implement big brain comparison since its a5 way and my code only takes 2
def determine_rank(hands,card_hierarchy):
    hand_hierarchy = {}
    # take 1, compare iteraively against rest, starting with strongest
    for i in range(len(hands)):
        winners = []
        rank = 0
        for hand in hands:
            winners.append(compare_hands(hands[i],hand,card_hierarchy))
#         print(f"winners for hand {hands[i]}: {winners}")
        for w in winners:
            # count number of times this hand won
            if w == hands[i]:
                rank += 1
        hand_hierarchy[hands[i]] = rank
    return hand_hierarchy        
        
    

In [8]:
# test w example
exhands = ['32T3K','T55J5','KK677','KTJJT','QQQJA']
determine_rank(exhands,card_values)
# lets go

{'32T3K': 1, 'T55J5': 4, 'KK677': 3, 'KTJJT': 2, 'QQQJA': 5}

In [9]:
# last part, get winnings
winnings = [765,684,28,220,483]
total_wins = 0
for i,k in enumerate(determine_rank(exhands,card_values)):
    total_wins += determine_rank(exhands,card_values)[k]*winnings[i]
total_wins

6440

In [10]:
# now to get the solution to part 1
d7input = []
with open('day7data.txt') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        d7input.append(line)
# kinda annoying i had to do this manually tbh

In [11]:
d7hands = []
d7bids = []
for line in d7input:
    splits = line.split(' ')
    hand = splits[0]
    bid = splits[1]
    d7hands.append(hand)
    d7bids.append(bid)

In [12]:
d7ranks = determine_rank(d7hands,card_values)

In [13]:
total_wins = 0
for i,k in enumerate(d7ranks):
    total_wins += d7ranks[k]*int(d7bids[i])
total_wins

250232501

# part 2

In [22]:
# part 2 bullshit
card_values2 = {'A': 100, 'K': 99, 'Q':98, 'J':1, 'T': 96, '9':95, '8':94, '7':93, '6':92, '5':91, '4':90, '3':89, '2':88}
#modify this fn too
def hand_type2(hand):
    h1 = [*hand]
    # can probably reuse a lot of this, just need to modify the J rule by turning it into most freq card
    if len(set(h1)) < 5 and 'J' in h1: # has atleast a pair and J
        # get max freq and highest card and turn into J
        hand_count = {i:h1.count(i) for i in h1}
        most_freq_card = max(hand_count, key=hand_count.get)
        # its ok if htere is a tie in this case since we are just comparing hand types
        h2 = []
        for card in h1:
            if card == 'J':
                h2.append(most_freq_card)
            else:
                h2.append(card)
        h1 = h2
    # first check 5 of kind
    if len(set(h1)) == 1:
        return 5
    elif len(set(h1)) == 2:
        # denotes both full house and 4 of kind so need to distinguish
        hand_count = {i:h1.count(i) for i in h1}
        # sort this dict by value
        if hand_count[max(hand_count, key=hand_count.get)] == 4:
            return 4
        else:
            return 3
    elif len(set(h1)) == 3:
        # need to distinguish between 3 of kind and 2 pair
        hand_count = {i:h1.count(i) for i in h1}
#         print(max(hand_count, key=hand_count.get))
        if hand_count[max(hand_count, key=hand_count.get)] == 3:
            return 2
        else:
            return 1
    elif len(set(h1)) == 4: # one pair
        return 0
    else: #high card
        return -1

In [23]:
hand_type2('KKQQJ')
# cash money

3

In [24]:
#now need to modify comparison - can prob use the same fn since we changed the hierarchy schema, but redoing it cuz im lazy
# now need to account for if you have same type, take ordering heirarchy
def compare_hands2(hand1,hand2, card_heirarchy):
    h1t = hand_type2([*hand1])
    h2t = hand_type2([*hand2])
#     print(f"hand type 1: {h1t} hand type 2: {h2t}")
    if h1t == h2t: #hands are same type
        diff_ind = next((i for i, (el1,el2) in enumerate(zip([*hand1], [*hand2])) if el1 != el2),None)
        if diff_ind is not None:
            #get those values and compare 
            h1v = [*hand1][diff_ind]
            h2v = [*hand2][diff_ind]
            if card_heirarchy[h1v] > card_heirarchy[h2v]:
                return hand1
            else:
                return hand2
        else:
            return hand1
    elif h1t > h2t:
        return hand1
    else: 
        return hand2

In [32]:
compare_hands2('QJQQ2','QQQQ2',card_values2)
#ezpz

'QQQQ2'

In [26]:
#ez modify, lazy execution
def determine_rank2(hands,card_hierarchy):
    hand_hierarchy = {}
    # take 1, compare iteraively against rest, starting with strongest
    for i in range(len(hands)):
        winners = []
        rank = 0
        for hand in hands:
            winners.append(compare_hands2(hands[i],hand,card_hierarchy))
#         print(f"winners for hand {hands[i]}: {winners}")
        for w in winners:
            # count number of times this hand won
            if w == hands[i]:
                rank += 1
        hand_hierarchy[hands[i]] = rank
    return hand_hierarchy        
        

In [27]:
# ok now work in the example scenario
exrank2 = determine_rank2(exhands,card_values2)
exrank2

{'32T3K': 1, 'T55J5': 3, 'KK677': 2, 'KTJJT': 5, 'QQQJA': 4}

In [28]:
# last part, get winnings
winnings = [765,684,28,220,483]
total_wins = 0
for i,k in enumerate(exrank2):
    total_wins += exrank2[k]*winnings[i]
total_wins
#beastmode

5905

In [39]:
# final answer
d7ranks2 = determine_rank2(d7hands,card_values2)
total_wins = 0
for i,k in enumerate(d7ranks2):
    total_wins += d7ranks2[k]*int(d7bids[i])
total_wins

249138943